In [0]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

{}

In [0]:
# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# Google Colabでデータセットをダウンロード
!pip install kaggle
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 51.5MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.8MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.91MB/s]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data

ModuleNotFoundError: No module named 'pandas'

In [2]:
# data_folder = '../data/'  # Local環境
data_folder = ''    #Google Colab
train = pd.read_csv(data_folder + 'train.csv')
test = pd.read_csv(data_folder + 'test.csv')

# テストデータの'PassengerId'を取得しておく
passenger_id = test.loc[:, 'PassengerId']

# 表示
train

NameError: name 'pd' is not defined

In [3]:
# 'Cabin"のデータはNanを0、それ以外を1に置き換える
def cabin_encode(data):
    data['Cabin'] = data['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)
    return data


# カテゴリ変数をLabel Encoding
def labeling(data, cat_cols):
    for cat in cat_cols:
        cat_i = data.columns.get_loc(cat)
        labels, _ = pd.factorize(data.iloc[:, cat_i])
        for i in range(len(labels)):
            data.iat[i, cat_i] = labels[i]

    return data


def standard(data, num_cols):
    # 数値データのNanを0に置き換える
    data[num_cols] = data[num_cols].fillna(0)
    # 数値データを正規化
    scaler = StandardScaler()
    data[num_cols] = scaler.fit_transform(data[num_cols])

    return data


ans_col = ['Survived']
cat_cols = ['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
num_cols = list(set(train.columns) - set(ans_col + cat_cols))
train = cabin_encode(train)
train = labeling(train, cat_cols)
train = standard(train, num_cols)

cat_cols = ['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
num_cols = list(set(test.columns) - set(cat_cols))
test = cabin_encode(test)
test = labeling(test, cat_cols)
test = standard(test, num_cols)

NameError: name 'train' is not defined

In [4]:
# IDと名前はユニークなデータなので削除
# Ticketはダブリが少ないので削除
del_cols = ['PassengerId', 'Name', 'Ticket']

# トレーニングデータを分割
train_x = train.drop(del_cols + ans_col, axis=1)
train_y = train.loc[:, ans_col]

# テストデータの必要のないカラムを削除
test = test.drop(del_cols, axis=1)

# カテゴリ変数と正解データをOne Hot Encoding
cat_cols = list(set(cat_cols) - set(del_cols))
train_x = pd.get_dummies(train_x, columns=cat_cols)
train_y = pd.get_dummies(train_y, columns=ans_col)
test = pd.get_dummies(test, columns=cat_cols)

NameError: name 'train' is not defined

In [5]:
# トレインデータ確認
train_x

NameError: name 'train_x' is not defined

In [6]:
# テストデータ確認
test

NameError: name 'test' is not defined

In [7]:
# トレインデータに'Embarked_-1'があったので列を削除
train_x = train_x.drop('Embarked_-1', axis=1)

NameError: name 'train_x' is not defined

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def to_cuda(obj):
    if torch.cuda.is_available():
        return obj.cuda()
    return obj

NameError: name 'nn' is not defined

In [9]:
input_size = len(train_x.columns)
outpu_size = len(train_y.columns)

train_x = torch.FloatTensor(np.array(train_x))
train_x = to_cuda(train_x)
train_y = torch.FloatTensor(np.array(train_y))
train_y = to_cuda(train_y)

NameError: name 'train_x' is not defined

In [10]:
HIDDEN_SIZE = 32
EPOCH_NUM = 300
BATCH_SIZE = 100

train = data.TensorDataset(train_x, train_y)
train_loader = data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

net = Net(input_size, HIDDEN_SIZE, outpu_size)
net = to_cuda(net)
print(net)

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

NameError: name 'data' is not defined

In [11]:
# 学習
loss_log = []
for e in range(EPOCH_NUM):
    total_loss = 0
    for x, y in train_loader:
        x, y = Variable(x), Variable(y)
        optimizer.zero_grad()
        output = net(x)

        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    ave_loss = total_loss / BATCH_SIZE
    loss_log.append(ave_loss)

    if (e + 1) % 50 == 0:
        print('Epoch: %d | Avearage Loss: %d' % (e + 1, ave_loss))

plt.plot([i for i in range(len(loss_log))], loss_log)

NameError: name 'train_loader' is not defined

In [12]:
test = torch.FloatTensor(np.array(test))
test = to_cuda(test)
test = Variable(test)

net.eval()
result = torch.max(net(test).data, 1)[1]

NameError: name 'torch' is not defined

In [13]:
# Google ColabでSubmit用ファイル保存
submission = pd.DataFrame()
submission['PassengerId'] = passenger_id
submission['Survived'] = result.data.numpy()
submission.to_csv('submission.csv', index=False)

NameError: name 'pd' is not defined

kaggleのsubmit最高スコアは77.598%